In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [3]:
import torch
from torch.utils.data import DataLoader

from data.scannet.utils_scannet import ScanNetDataset
from matching.gmflow_dense.gmflow_dense import GMflowDensePoseDeterministic
from flow_regressors.regressors import DensePoseRegressorV7

from training.loss_pose import LossPose
from training.loss_deterministic import DeterministicLossMixed
from training.train_deterministic import train, validate
from utils.model import load_checkpoint

from tqdm.auto import tqdm
import wandb

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Data

In [4]:
train_data = ScanNetDataset(
    root_dir='/home/project/data/ScanNet/scans/',
    npz_path='/home/project/code/data/scannet_splits/smart_sample_train_ft.npz',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz',
    calculate_flow=True
)

train_loader = DataLoader(train_data, batch_size=2, shuffle=True, drop_last=True, pin_memory=True, num_workers=0)

val_data = ScanNetDataset(
    root_dir='/home/project/data/ScanNet/scans/',
    npz_path='/home/project/code/data/scannet_splits/smart_sample_val.npz',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz',
    calculate_flow=True
)

val_loader = DataLoader(val_data, batch_size=2, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

### 2. Configuration

In [5]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

config = dict(
    general = dict(
        experiment_name='DETERMINISTIC_FT',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        
        n_epochs=5,
        n_steps_per_epoch=len(train_loader.dataset)//train_loader.batch_size,
        n_accum_steps=16,
        batch_size=train_loader.batch_size,
        
        swa=False,
        n_epochs_swa=None,
        n_steps_between_swa_updates=None,

        repeat_val_epoch=1,
        repeat_save_epoch=1,
          
        model_save_path='../../src/weights/DETERMINISTIC_FT'
    )
)

### 3. Model

In [6]:
checkpoint_gmflow = load_checkpoint('/home/project/code/src/matching/gmflow/weights/pretrained/gmflow_with_refine_kitti-8d3b9786.pth',
                             config['general']['device'])

checkpoint_regressor = load_checkpoint('/home/project/code/src/weights/COARSE_FLOW_EXP_1_4.pth',
                             config['general']['device'])

In [7]:
regressor = DensePoseRegressorV7(init_loss_weights=[0.0, -3.0, -3.0])
regressor.load_state_dict(checkpoint_regressor['model'])

model = GMflowDensePoseDeterministic(regressor, fine_tuning=False)
model.flow_module.load_state_dict(checkpoint_gmflow['model'])

model.to(torch.float32)
model.to(config['general']['device']);

In [8]:
for name, p in model.named_parameters():
    if any(p_name in name for p_name in
          ('transformer.layers.5', 'feature_flow_attn', 'upsampler', 'decoder')):
        p.requires_grad = True
    else:
        p.requires_grad = False

In [9]:
# data = next(iter(train_loader))

In [10]:
# for key in data.keys():
#     if key in ('image0', 'image1', 'K0', 'K1', 'flow_0to1', 'mask'):
#         data[key].to('cpu')

# with torch.no_grad():
#     preds, q, t = model(
#         data['image0'],
#         data['image1'],
#         attn_splits_list=[2, 8],
#         corr_radius_list=[-1, 4],
#         prop_radius_list=[-1, 1],
#         K_q=data['K0'],
#         K_s=data['K1'],
#         scales_q=(0.25, 0.25),
#         scales_s=(0.25, 0.25))

In [11]:
# loss = train_loss(data['flow_0to1'], data['mask'], data['T_0to1'], preds['flow_preds'], q, t, model.dense_pose_regressor.loss_weights)

In [12]:
# loss.requires_grad

In [13]:
# for name, p in model.named_parameters():
#     try:
#         print(name, p.data.dtype)
#     except:
#         continue

In [14]:
# p.data

In [15]:
# (torch.abs(data['flow_0to1'] - preds['flow_preds'][-2]) * data['mask']) .mean()

In [16]:
# preds['flow_preds'][-1].shape

In [17]:
# for key in data.keys():
#     if key in ('image0', 'image1', 'K0', 'K1', 'flow_0to1', 'mask'):
#         data[key].to('cpu')


In [18]:
# data['flow_0to1'].shape

### 4. Loss, optimizer, scheduler

In [19]:
train_loss = DeterministicLossMixed()
val_loss = LossPose()

In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
config['optimizer'] = optimizer.__dict__['defaults']

In [21]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                   step_size=2,
                   gamma=0.8)

In [22]:
# validate(model, val_loss, val_loader, config['general']['device'])

### 6. Experiment

In [ ]:
train(model, optimizer, scheduler, train_loss, val_loss, train_loader, val_loader, config, **config['general'])

wandb: Currently logged in as: kovanic. Use `wandb login --relogin` to force relogin


  0%|                                                                                                                             | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 16%|█████████████████▌                                                                                            | 7986/50000 [1:45:40<9:40:34,  1.21it/s]